<h2 style="color:green"> Feature Extraction from MP3 Files</h2>
Author: (Afif Al Mamun)[https://afifaniks.github.io]<br>
Date: 3 July, 2019

In [1]:
import librosa
import os
import csv
import pandas as pd
import numpy as np
import traceback
from tqdm import tqdm_notebook as tqdm

In [14]:
def get_files(dir_path):
    '''Returns the list of files from a specified directory'''
    file_list = []
    
    for file in os.scandir(dir_path):
        if file.is_file:
            file_list.append(file.name)
    
    return file_list

In [15]:
def load_mp3(file_path, start_time, duration_time):
    '''Loads an MP3 file as an array of samples'''
    file, sr = librosa.load(file_path, offset=start_time, duration=duration_time)
    
    return (file, sr)

In [16]:
def get_zero_crossing_rate(file):
    '''Returns number of zero crossings of an MP3 file in a specified time interval'''
    start_time = 0
    
    zero_crossings = librosa.zero_crossings(file[start_time:], pad=False)
    z_c = sum(zero_crossings)
    
    return z_c

In [17]:
def get_spectral_centroid(file):
    '''Returns mean value of Spectral Centroid of an MP3 file'''
    spectral_centroid = librosa.feature.spectral_centroid(file)
    return np.mean(spectral_centroid)

In [18]:
def get_spectral_rolloff(file):
    '''Returns mean value of Spectral Rolloff of an MP3 file'''
    spectral_rolloff = librosa.feature.spectral_rolloff(file)
    return np.mean(spectral_rolloff)

In [19]:
def get_mfcc(file):
    '''Returns mean values of Mel Frequency Cepstral Coefficients (which
    is 20 in length) of an MP3 file'''
    mfcc = librosa.feature.mfcc(file)
    
    result_mfcc = []
    
    for i in range(mfcc.shape[0]):
        result_mfcc.append(np.mean(mfcc[i]))

    return result_mfcc

In [20]:
def get_chroma_freq(file):
    '''Returns mean value of Chroma Frequencies of an MP3'''
    chroma_frequencies = librosa.feature.chroma_stft(file)
    
    return np.mean(chroma_frequencies)

In [21]:
def get_spectral_bandwidth(file):
    '''Returns mean value of Spectral Bandwidth of an MP3'''
    spectral_bandwidth = librosa.feature.spectral_bandwidth(file)
    
    return np.mean(spectral_bandwidth)

In [22]:
def get_rmse(file):
    '''Returns mean value of Root Mean Square Energy of an MP3 signal'''
    rmse = librosa.feature.rmse(file)
    return np.mean(rmse)

In [23]:
def get_delta(file):
    '''Returns delta of an MP3 signal'''
    delta = librosa.feature.delta(file)
    return np.mean(delta)

In [24]:
def get_melspectogram(file):
    '''Returns melspectogram of an MP3 signal'''
    melspectogram = librosa.feature.melspectrogram(file)
    return np.mean(melspectogram)

In [25]:
def get_tempo(file):
    '''Returns tempo of an MP3 file'''
    return librosa.beat.tempo(file)[0]

In [26]:
def create_csv():
    column_names = "file_name zero_crossing spectral_centroid spectral_rolloff spectral_bandwidth chroma_frequency rmse delta melspectogram tempo"
    
    for i in range(20):
        column_names += " mfcc{}".format(i)
    
    column_names += " label"
    
    with open('dataset.csv', mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(column_names.split())
    
    print("CSV File Created")   

In [27]:
def write_csv(file_name, 
              zero_crossing, 
              spectral_centroid,
              spectral_rolloff,
              spectral_bandwidth,
              chroma_frequency,
              rmse,
              delta,
              melspectogram,
              tempo,
              mfcc,
              label):
    
    if not os.path.isfile('dataset.csv'):
        create_csv()
    
    row_values = [file_name, zero_crossing, spectral_centroid, 
                  spectral_rolloff, spectral_bandwidth, chroma_frequency, rmse, delta, melspectogram, tempo]
    
    for i in range(20): # mfcc length is always 20
        row_values.append(mfcc[i])
    
    row_values.append(label)
    
    with open('dataset.csv', mode='a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(row_values)

In [28]:
def extract_features(starting_offset, duration):
    genre_dir = os.listdir('data') # Getting subdirectories of data directory

    for genre in genre_dir:        
        files = get_files('data/' + genre)
        
        files_iterator = tqdm(files, ncols=700) #ncols sets the progress bar width
        
        print("Processing files in directory: data/" + genre +"/")
        
        for file_name in files_iterator:
            files_iterator.set_description(file_name)
            try:
                if file_name.endswith('.mp3'):
                    
                    file, sample_rate = load_mp3('data/' + genre + '/' + file_name, starting_offset, duration)

                    zero_crossing = get_zero_crossing_rate(file)
                    spec_centroid = get_spectral_centroid(file)
                    spec_rolloff = get_spectral_rolloff(file)
                    spec_bandwidth = get_spectral_bandwidth(file)
                    chroma_frequency = get_chroma_freq(file)
                    rmse = get_rmse(file)
                    delta = get_delta(file)
                    melspectogram = get_melspectogram(file)
                    tempo = get_tempo(file)
                    mfcc = get_mfcc(file)

                    write_csv(file_name,
                             zero_crossing,
                             spec_centroid,
                             spec_rolloff,
                             spec_bandwidth,
                             chroma_frequency,
                             rmse,
                             delta,
                             melspectogram, 
                             tempo,
                             mfcc, # (20, )
                             genre)
            except:
                print("Something went wrong while processing file: data/" + genre + "/" + file_name )
                print(traceback.print_exc())
                continue


In [29]:
# extract_features(20, 70)

In [30]:
# Printing extracted dataset
df = pd.read_csv("dataset.csv", encoding='latin-1')
pd.options.display.max_columns = None
df.head()

,file_name,zero_crossing,spectral_centroid,spectral_rolloff,spectral_bandwidth,chroma_frequency,rmse,delta,melspectogram,tempo,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,label
0,(Tomake Paowa) - Hridoy Khan & Tasnim Anika F...,125424,1770.889394,3893.575094,0.315795,0.315795,0.222552,-7.661680e-08,12.977708,172.265625,-107.906257,116.494382,23.320857,17.535451,9.382846,6.854393,-0.293704,-0.007064,-7.914926,1.532803,-5.300817,3.164991,-3.867179,5.325016,1.176338,-0.788755,-6.343288,2.488148,0.047027,7.211918,adhunik
1,008. Jai Bole Jete Naire - Jewel (1980).mp3,308658,2549.349989,4986.658880,0.310113,0.310113,0.156930,8.893313e-08,5.118271,143.554688,-68.452095,81.151777,-35.804154,44.550898,-17.021686,18.244996,-18.741383,12.376125,-16.116041,8.450346,-11.745541,8.358729,-12.691169,11.393858,-6.176215,9.947330,-6.987708,7.935905,-9.452455,1.582699,adhunik
2,03. Doob Habib ft Mousumi & Mosharraf Karim...,158328,2013.718396,4573.595848,0.301663,0.301663,0.093736,1.340824e-07,2.497263,103.359375,-193.043916,99.472065,21.751411,21.653152,7.772928,3.184855,-0.963999,0.415176,-12.447337,-2.746030,-8.932615,-3.361615,-6.730814,0.184162,-4.139533,4.409801,3.190835,4.862090,-6.406590,-0.705287,adhunik
3,034. Mukhe Bolo Valobashi by Muni Kis.mp3,136711,1827.495303,4215.075766,0.349875,0.349875,0.039457,-1.908483e-08,0.421975,135.999178,-264.475953,110.057377,7.935531,29.494746,0.908531,13.580350,-2.774143,8.508934,-0.374241,1.220914,5.370642,8.563364,-7.990767,3.611352,-4.211268,5.490051,-7.556331,1.124302,-4.306443,0.957663,adhunik
4,(Tomake Paowa) - Hridoy Khan & Tasnim Anika F...,91851,1823.247891,4057.641091,0.292992,0.292992,0.217565,-4.354582e-07,12.582756,172.265625,-111.047111,111.526984,25.171742,16.971460,6.419625,6.483828,-2.587302,-1.135894,-11.046108,1.804448,-4.221120,3.367069,-4.470812,4.747750,-0.102839,-2.103499,-6.056158,2.846886,-1.442194,7.619524,adhunik
